<a href="https://colab.research.google.com/github/Sonal-kumar1998/ANN_Based_Device_Modelling/blob/main/Data_preprocessing_DC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



#Import library


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np

#Conversion function for terminal current and charge:

In [3]:
def conversion_function(I_ds, V_ds):
  I_0 = 10**(-12)          # 1 picometer
  y1 = I_ds/(I_0*V_ds)
  y = np.log10(y1)
  return y

# TCAD dataset  

In [4]:
TCAD_data_path = '/content/drive/MyDrive/DDP/Dataset/DC_dataset/tcad_data_set.csv'
TCAD_data = pd.read_csv(TCAD_data_path)
print(f"The shape of the TCAD dataset is {TCAD_data.shape}" )
TCAD_data.head()

The shape of the TCAD dataset is (1047654, 5)


,"#""gate OuterVoltage""",drain OuterVoltage,drain TotalCurrent,substrate OuterVoltage,source OuterVoltage
0,0.0,-3.430000e-35,5.390000e-16,-1.0,-1.810000e-36
1,0.0,8.000000e-03,4.270000e-09,-1.0,1.580000e-34
2,0.0,1.600000e-02,7.040000e-09,-1.0,-4.780000e-33
3,0.0,2.400000e-02,8.790000e-09,-1.0,3.970000e-34
4,0.0,3.200000e-02,9.880000e-09,-1.0,4.760000e-34


#Preprocess TCAD dataset to convert ANN data set

In [5]:
#TCAD_data_path = "/content/drive/MyDrive/DDP/Dataset/tcad_data_set.csv"
TCAD_data = pd.read_csv(TCAD_data_path)
#print(TCAD_data.shape)   

############# remove the header of data_set during concatination
TCAD_data = TCAD_data[TCAD_data['#"gate OuterVoltage"'] != '#"gate OuterVoltage"']   
#print(TCAD_data.shape)

# convert the data set to float type 
TCAD_data = TCAD_data.astype("float") 

############ remove invaild data point  for dataset
TCAD_data["Vailid"] = TCAD_data['drain TotalCurrent'] * TCAD_data["drain OuterVoltage"]
TCAD_data = TCAD_data[TCAD_data["Vailid"] >= 0]
TCAD_data = TCAD_data.drop(columns="Vailid")
#print(TCAD_data.shape)

########### preprocess the TCAD_dataset
I_ds = TCAD_data['drain TotalCurrent'] 
V_ds = TCAD_data['drain OuterVoltage']
TCAD_data["V_gs"] = TCAD_data['#"gate OuterVoltage"'] -TCAD_data["source OuterVoltage"] 
TCAD_data["V_ds"] = TCAD_data["drain OuterVoltage"] -TCAD_data["source OuterVoltage"] 
TCAD_data["V_bs"] = TCAD_data["substrate OuterVoltage"] -TCAD_data["source OuterVoltage"] 
TCAD_data["converted_Ids"] = conversion_function(I_ds, V_ds)

########### Save the data set as ANN_data_set
ANN_data_set = TCAD_data
ANN_data_set.to_csv('/content/drive/MyDrive/DDP/Dataset/DC_dataset/ann_data_set.csv')
# note the TCAD_data_set is in str format and ANN_data_set in float dataset
print(f"The shape of the ANN dataset is {ANN_data_set.shape}" )
ANN_data_set.head()

The shape of the ANN dataset is (1047274, 9)


,"#""gate OuterVoltage""",drain OuterVoltage,drain TotalCurrent,substrate OuterVoltage,source OuterVoltage,V_gs,V_ds,V_bs,converted_Ids
1,0.0,0.008,4.270000e-09,-1.0,1.580000e-34,-1.580000e-34,0.008,-1.0,5.727338
2,0.0,0.016,7.040000e-09,-1.0,-4.780000e-33,4.780000e-33,0.016,-1.0,5.643453
3,0.0,0.024,8.790000e-09,-1.0,3.970000e-34,-3.970000e-34,0.024,-1.0,5.563778
4,0.0,0.032,9.880000e-09,-1.0,4.760000e-34,-4.760000e-34,0.032,-1.0,5.489607
5,0.0,0.040,1.050000e-08,-1.0,5.160000e-34,-5.160000e-34,0.040,-1.0,5.419129


#Unique is function which return dictionary of unique value

In [6]:
# input argument global path variable
# return dictnary of the unique value
global ann_datapath
ann_datapath = "/content/drive/MyDrive/DDP/Dataset/DC_dataset/ann_data_set.csv"
def unique(path):
  value = {}
  data = pd.read_csv(path)
  data = data[data['#"gate OuterVoltage"'] != '#"gate OuterVoltage"']
  unique_gate_volt = data['#"gate OuterVoltage"'].unique()
  unique_gate_volt = unique_gate_volt.reshape(unique_gate_volt.shape[0],1)
  unique_drain_volt = data['drain OuterVoltage'].unique()
  unique_drain_volt = unique_drain_volt.reshape(unique_drain_volt.shape[0],1)
  value["drain_voltage"] = unique_drain_volt.reshape(1,unique_drain_volt.shape[0])
  value["gate_voltage"] =np.sort(unique_gate_volt.reshape(1,unique_gate_volt.shape[0]))
  return value

In [7]:
print(unique(ann_datapath)["drain_voltage"][:,:5]) # print first 3 unique drain volatge
print(unique(ann_datapath)["gate_voltage"][:, :5]) # print first 3 unique gate_voltage 

[[0.008 0.016 0.024 0.032 0.04 ]]
[[0.   0.02 0.04 0.06 0.08]]


# Split the data into training and testing data_set

In [8]:
unique_gate = unique(ann_datapath)["gate_voltage"]             # find the unique gate voltage
gate_voltage = unique_gate.tolist()[0]                         # list of gate voltage in ann_data_set
test_gate_voltage = []
for i in range(5, len(gate_voltage) ,10):
  test_gate_voltage.append(float(unique_gate[:,i]))  

## list of test gate voltage
training_gate_volatge = newlist = [volt for volt in gate_voltage if volt not in test_gate_voltage]

########################__________Loading_data_set__________#############################
ANN_data_set = pd.read_csv(ann_datapath)
ANN_data_set = ANN_data_set.drop(columns="Unnamed: 0")

########################__________Testing_data_set__________#############################
Test_data_set = ANN_data_set.loc[ANN_data_set['#"gate OuterVoltage"'].isin(test_gate_voltage)]
Test_data_set.to_csv('/content/drive/MyDrive/DDP/Dataset/DC_dataset/test_data_set.csv')   # save the testing data set

########################__________Training_data_set__________#############################
Training_data_set = ANN_data_set.loc[ANN_data_set['#"gate OuterVoltage"'].isin(training_gate_volatge)]
Training_data_set.to_csv('/content/drive/MyDrive/DDP/Dataset/DC_dataset/training_data_set.csv')

print(f"The shape of the Training dataset is {Training_data_set.shape}" )
Training_data_set.head()
ANN_data_set.head()

The shape of the Training dataset is (942593, 9)


,"#""gate OuterVoltage""",drain OuterVoltage,drain TotalCurrent,substrate OuterVoltage,source OuterVoltage,V_gs,V_ds,V_bs,converted_Ids
0,0.0,0.008,4.270000e-09,-1.0,1.580000e-34,-1.580000e-34,0.008,-1.0,5.727338
1,0.0,0.016,7.040000e-09,-1.0,-4.780000e-33,4.780000e-33,0.016,-1.0,5.643453
2,0.0,0.024,8.790000e-09,-1.0,3.970000e-34,-3.970000e-34,0.024,-1.0,5.563778
3,0.0,0.032,9.880000e-09,-1.0,4.760000e-34,-4.760000e-34,0.032,-1.0,5.489607
4,0.0,0.040,1.050000e-08,-1.0,5.160000e-34,-5.160000e-34,0.040,-1.0,5.419129


In [9]:
print(Test_data_set.shape)
print(f"The shape of the Test dataset is {Test_data_set.shape}" )
Test_data_set.head()

(104681, 9)
The shape of the Test dataset is (104681, 9)


,"#""gate OuterVoltage""",drain OuterVoltage,drain TotalCurrent,substrate OuterVoltage,source OuterVoltage,V_gs,V_ds,V_bs,converted_Ids
2251,0.1,0.008,6.500000e-08,-1.0,5.220000e-30,0.1,0.008,-1.0,6.909823
2252,0.1,0.016,1.090000e-07,-1.0,5.220000e-30,0.1,0.016,-1.0,6.833307
2253,0.1,0.024,1.390000e-07,-1.0,-1.270000e-36,0.1,0.024,-1.0,6.762804
2254,0.1,0.032,1.590000e-07,-1.0,5.220000e-30,0.1,0.032,-1.0,6.696247
2255,0.1,0.040,1.710000e-07,-1.0,-1.270000e-36,0.1,0.040,-1.0,6.630936
